In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
from coeditor.dataset import *
from coeditor.encoding import *
from spot.utils import pretty_print_dict

os.chdir(proj_root())

In [2]:
# encoder = AnalysisBasedEditEncoder(extra_ctx_names=("usees", "post-usees"))
encoder = FileBasedEditEncoder()
data = dataset_from_projects([proj_root()], encoder, max_history_per_repo=20)

Create tokenized edits: 100%|██████████| 1/1 [00:16<00:00, 16.15s/chunk]


In [4]:
ex=data.all_edits()[1]
# print(ex.elems)
print(ex.show())

========Ground Truth========
 <0>:<add> def collect_annotations(
     <add>     code: cst.CSTNode,
     <add> ) -> Tuple[list[AnnotPath], dict[AnnotPath, cst.Annotation]]:
     <del> def collect_annotations(code: cst.CSTNode) -> Dict[AnnotPath, Optional[cst.Annotation]]:

 <3>:<add>     return collector.annot_paths, collector.annotations
     <del>     return collector.annotations

========Main Code========
<extra_id_0>def collect_annotations(code: cst.CSTNode) -> Dict[AnnotPath, Optional[cst.Annotation]]:
<extra_id_1>    collector = AnnotCollector()
<extra_id_2>    code.visit(collector)
<extra_id_3>    return collector.annotations
<extra_id_4>

===========left context===========
from dataclasses import dataclass
from distutils.log import error
 <add> from logging import warn
from posixpath import dirname, realpath
import re
from typing import Iterable
from.utils import *
import subprocess


 <add> @dataclass(order=True, unsafe_hash=True)
 <del> @dataclass(frozen=True)
class AnnotPath:

In [16]:
# dataset_name = "SPOT"
dataset_name = "medium"
window = WindowArgs(4096)
encoder = AnalysisBasedEditEncoder(
    window=window, extra_ctx_min_size=1000, extra_ctx_names=("usees", "post-usees")
)
save_dir = get_dataset_dir(dataset_name) / "tokenized-file_collapsed"# repr_modified_args(encoder)
datasets = load_datasets(save_dir)

In [17]:
for group, dataset in datasets.items():
    print("=" * 20, group, "=" * 20)
    pretty_print_dict(dataset.overall_stats())

==================== test ====================
n_projects: 20
n_edits: 12666
input_size:
   mean: 1838.3
   median: 1537.5
   min: 15
   max: 4096
output_size:
   mean: 113.59
   median: 75
   min: 5
   max: 4776
==================== valid ====================
n_projects: 10
n_edits: 2297
input_size:
   mean: 1770.5
   median: 1541
   min: 19
   max: 4096
output_size:
   mean: 116.91
   median: 89
   min: 5
   max: 1567
==================== train ====================
n_projects: 100
n_edits: 50804
input_size:
   mean: 1819.5
   median: 1489
   min: 19
   max: 4096
output_size:
   mean: 97.962
   median: 64
   min: 5
   max: 8434


In [14]:
for group, dataset in datasets.items():
    print("=" * 20, group, "=" * 20)
    pretty_print_dict(dataset.overall_stats())

==================== test ====================
n_projects: 20
n_edits: 12667
input_size:
   mean: 1773.5
   median: 1648
   min: 19
   max: 4102
output_size:
   mean: 113.57
   median: 75
   min: 5
   max: 4776
==================== valid ====================
n_projects: 10
n_edits: 2297
input_size:
   mean: 1702.5
   median: 1559
   min: 23
   max: 4102
output_size:
   mean: 116.91
   median: 89
   min: 5
   max: 1567
==================== train ====================
n_projects: 100
n_edits: 50792
input_size:
   mean: 1700.8
   median: 1559
   min: 23
   max: 4102
output_size:
   mean: 97.974
   median: 64
   min: 5
   max: 8434


In [7]:
with_ctx = [e for e in data.all_edits() if decode_tokens(AnalysisBasedEditEncoder.CtxSepTokens) in decode_tokens(e.input_tks)]

print("All edits:", len(data.all_edits()))
print("All edits with extra context:", len(with_ctx))

All edits: 1331
All edits with extra context: 360


In [8]:
print(with_ctx[4].show())

========Left Context========

class UsageAnalysis:
 <add>     def __init__(self, project: PythonProject, add_override_usages: bool):
 <del>     def __init__(self, project: PythonProject):
        self.project = project
 <add>         self.add_override_usages = add_override_usages
        self.ns_hier = ModuleHierarchy.from_modules(project.modules.keys())
        module2ns = build_project_namespaces(project)
        self.sorted_modules = list(module2ns.keys())

        self.path2elem = {v.path: v for v in project.all_elems()}
        self.path2class = {
            cls.path: cls
            for mod in project.modules.values()
            for cls in mod.all_classes()
        }

        for mname, ns in module2ns.items():
            for s, p in ns.items():
                if p in self.path2class:
                    cls = self.path2class[p]
                    self.path2class.setdefault(ProjectPath(mname, s), cls)
                elif p in self.path2elem:
                    self.path2el

In [16]:
dataset_name = "medium"
save_dir = get_dataset_dir(dataset_name) / "tokenized-file_collapsed"
datasets = load_datasets(save_dir)

In [17]:
for group, dataset in datasets.items():
    print("=" * 20, group, "=" * 20)
    pretty_print_dict(dataset.overall_stats())

==================== test ====================
n_projects: 20
n_edits: 12666
input_size:
   mean: 1838.3
   median: 1537.5
   min: 15
   max: 4096
output_size:
   mean: 113.59
   median: 75
   min: 5
   max: 4776
==================== valid ====================
n_projects: 10
n_edits: 2297
input_size:
   mean: 1770.5
   median: 1541
   min: 19
   max: 4096
output_size:
   mean: 116.91
   median: 89
   min: 5
   max: 1567
==================== train ====================
n_projects: 100
n_edits: 50804
input_size:
   mean: 1819.5
   median: 1489
   min: 19
   max: 4096
output_size:
   mean: 97.962
   median: 64
   min: 5
   max: 8434


In [ ]:
for group in datasets:
    if not (save_dir / f"{group}.pkl").exists():
        continue
    dataset = pickle_load(save_dir / f"{group}.pkl")
    print("=" * 20, group, "=" * 20)
    display(dataset.per_repo_stats())

In [ ]:
print(list(datasets["test"].all_edits())[1].show_prediction())